In [1]:
import copy
from datetime import datetime
 
# Получаем индекс элемента в массиве
# Если элемент не найден, добавляем его в конец массива и возвращаем его номер
def getIndex(arr, val):
  try:
    return arr.index(val)
  except:
    arr.append(val)
    return len(arr) - 1
 
# Из переданного массива arr получаем запись, у которой поле key соответствует значению value
# Если такая запись не обнаружена, то создаем новую запись из объекта default, устанавливаем полу key в значение value и помещаем в массив
def getValueOrDefault(arr, key, value, default):
  res = next((x for x in arr if x[key] == value), None)
  if (res == None):
    res = copy.deepcopy(default)
    res[key] = value
    arr.append(res)
  return res
 
# Из строки даты (формат: дд.мм.гггг) получаем месяц
def getBdate(bdate):
  return datetime.strptime(bdate, '%d.%m.%Y').month

In [2]:
import numpy as np
import csv
import sys
 
input = []
 
with open('data/input.csv', newline='') as csvfile:
  inputreader = csv.reader(csvfile, delimiter=',', quotechar='"')
  for row in inputreader:
    input.append(row)
 
# Прекращаем работу приложения, если не было прочитано ни одной строки
# или первая строка (заголовок) не содержит ни одного столбца 
if (len(input) < 2):
  print('Датасет пустой. Приложение будет остановлено')
  sys.exit()
 
if (len(input[0]) == 0):
  print('Датасет пустой. Приложение будет остановлено')
  sys.exit()
 
# Выводим размер датасета. 
# len - 1, потому что первый столбец - это заголовок.
print('Датасет содержит %d записей' % (len(input) - 1))
 
# Объявляем переменные, в которые запишем номера столбцов. 
# Если в датасети номера столбцов поменяются местами или пропадут, мы автоматические это определим
# значение по умолчанию -1, т.к. нумерация массива с 0. -1 означает, что столбца нет в датасете
npp = fio = sex = bdate = ege = discip = school = city = prof = budget = -1
 
# Заполняем номера столбцов, сравнивая их с названием ячейки из первой строки датасета
for j in range(len(input[0])):
  col = input[0][j]
  if (col == '№п/п'): npp = j
  if (col == 'ФИО'): fio = j
  if (col == 'Пол'): sex = j
  if (col == 'Дата рождения'): bdate = j
  if (col == 'Баллы ЕГЭ'): ege = j
  if (col == 'Предметы ЕГЭ'): discip = j
  if (col == 'Учебное заведение'): school = j
  if (col == 'Населенный пункт по прописке'): city = j
  if (col == 'Специальность/направление'): prof = j
  if (col == 'Основание поступления'): budget = j
 
if (fio == -1):
  print('Столбец ФИО не определен в датасете. Приложение будет остановлено')
  sys.exit()
 
discipList = []
schoolList = []
cityList = []
profList = []
budgetList = []
peopleList = []
 
# Объект человека по умолчанию
DEFAULT_PEOPLE = {'fio':None,'sex':None,'bdate':None,'city':None,'school':None,
                  'egeMathem':0,'egeRussky':0,'egeInform':0,'egePhysic':0,'egeSociety':0,
                  'prof':None,'budget':None}
 
# Проходим по датасету и заполняем объект человека информацией. 
# т.к. в датасете каждая строя - это бал ЕГЭ, то на одного человека несколько строк в датасете
# поэтому их надо объединить в один объект
# при этом заменяем строковые поля характерным номером
for j in range(1, len(input)):
  p = getValueOrDefault(peopleList, 'fio', input[j][fio], DEFAULT_PEOPLE)
  
  if (sex != -1):
    if (input[j][sex] == 'Мужской'):
      p['sex'] = 0
    elif (input[j][sex] == 'Женский'):
      p['sex'] = 1
    else:
      print('ВНИМАНИЕ: Неизвестный пол %s' % input[j][sex])
 
  if (bdate != -1):
    p['bdate'] = getBdate(input[j][bdate])
 
  if (city != -1):
    p['city'] = getIndex(cityList, input[j][city])
 
  if (school != -1):
    p['school'] = getIndex(schoolList, input[j][school])
 
  if (prof != -1):
    p['prof'] = getIndex(profList, input[j][prof])
 
  if (budget != -1):
    p['budget'] = getIndex(budgetList, input[j][budget])
 
  if (ege != -1 and discip != -1):
    if (input[j][discip] == 'Русский язык'):
      p['egeRussky'] = int(input[j][ege])
    elif (input[j][discip] == 'Математика'):
      p['egeMathem'] = int(input[j][ege])
    elif (input[j][discip] == 'Физика'):
      p['egePhysic'] = int(input[j][ege])
    elif (input[j][discip] == 'Информатика и ИКТ'):
      p['egeInform'] = int(input[j][ege])
    elif (input[j][discip] == 'Обществознание'):
      p['egeSociety'] = int(input[j][ege])
    else:
      print('ВНИМАНИЕ: Неизвестный предмет ЕГЭ %s' % input[j][discip])
 
print('Всего человек в датасете: %d' % len(peopleList))
 
 
 
# Подготавливаем датасет для нейронной сети
# Для этого все значения необходимо привести к диапазону 0..1
# Баллы ЕГЭ делим на 100, т.к. 100 максимальный бал,
# город и школу делим на количество записей
# профессии мы не делим, т.к. это число у нас определяет номер активируемого нейрона
# дату рождения (номер месяца) делим на 12 (кол-во месяцев в году)
# пол при заполнении уже лежит в диапазоне 0..1 (0 - мужской, 1 - женский)
 
nnDataset = np.array([])
nnLabels = np.array([])
 
# Количество нейронов на входном слое
# При заполнении массива nnDataset необходимо, чтобы количество параметров совпадало с этим числом
INPUT_SIZE = 9
 
for j in range(len(peopleList)):
  p = peopleList[j]
  # nnDataset прям скопировать, преобразуем список, туда запихиваем функцию предикта
  nnDataset = np.append(nnDataset, [
                    p['egeRussky'] / 100, 
                    p['egeMathem'] / 100, 
                    p['egePhysic'] / 100,
                    p['egeInform'] / 100,
                    p['egeSociety'] / 100,
                    p['sex'],
                    p['bdate'] / 12,
                    p['city'] / len(cityList),
                     p['school'] / len(schoolList)
                    ])
  
  nnLabels = np.append(nnLabels, p['prof'])
 
# приводим датасет к двумерному виду. 
nnDataset = nnDataset.reshape(-1, INPUT_SIZE)
 
train = copy.deepcopy(nnDataset)
train_labels = copy.deepcopy(nnLabels)

Датасет содержит 1310 записей
Всего человек в датасете: 443


In [3]:
# TensorFlow и tf.keras
import tensorflow as tf
from tensorflow import keras
 
model_first = keras.Sequential([
  #  слои модели    
  # Flatten преобразует в одномерный массив
  # keras.layers.Flatten(input_shape=(maxShape, 13)),
 
  keras.layers.InputLayer(INPUT_SIZE),
 
  keras.layers.Dense(12, activation='relu'),
  keras.layers.Dense(24, activation='relu'),
  keras.layers.Dense(12, activation='relu'),
 
  # Последний слой определяет профессию человека
  # Количество нейронов должно соответствовать количеству профессий
  keras.layers.Dense(len(profList), activation='softmax')
])
 
model_first.compile(optimizer=keras.optimizers.SGD(learning_rate=0.05, momentum=.5, nesterov=True),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
 
model_first.fit(train, train_labels, epochs=5000, batch_size=32, verbose=1)
 
good = 0
#вот этим проверять данные
for i in range(len(train)):
  checkDigit = (np.expand_dims(nnDataset[i],0))
  predictions_single = model_first.predict(checkDigit)#выход с последнего слоя, тк стоит софтмакс то выходы в сумме должны дать 1-цу
  predictProf = np.argmax(predictions_single[0])
  # print('Predict %d. Real: %d' % (predictProf, nnLabels[i]))
  if (predictProf == nnLabels[i]):
    good += 1
 
print('Accuracy %.4f' % (good / len(train)))

Epoch 1/5000
14/14 [==============================] - 0s 2ms/step - loss: 3.0084 - accuracy: 0.0528
Epoch 2/5000
14/14 [==============================] - 0s 2ms/step - loss: 2.9014 - accuracy: 0.1741
Epoch 3/5000
14/14 [==============================] - 0s 1ms/step - loss: 2.7308 - accuracy: 0.1809
Epoch 4/5000
14/14 [==============================] - 0s 1ms/step - loss: 2.5716 - accuracy: 0.1972
Epoch 5/5000
14/14 [==============================] - 0s 1ms/step - loss: 2.4759 - accuracy: 0.1524
Epoch 6/5000
14/14 [==============================] - 0s 815us/step - loss: 2.2701 - accuracy: 0.2265
Epoch 7/5000
14/14 [==============================] - 0s 962us/step - loss: 2.2370 - accuracy: 0.2268
Epoch 8/5000
14/14 [==============================] - 0s 1ms/step - loss: 2.1559 - accuracy: 0.2883
Epoch 9/5000
14/14 [==============================] - 0s 951us/step - loss: 2.0636 - accuracy: 0.2738
Epoch 10/5000
14/14 [==============================] - 0s 2ms/step - loss: 2.1138 - accuracy: 

14/14 [==============================] - 0s 1ms/step - loss: 1.5517 - accuracy: 0.3802
Epoch 83/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.6136 - accuracy: 0.3759
Epoch 84/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.5751 - accuracy: 0.3902
Epoch 85/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.6054 - accuracy: 0.3698
Epoch 86/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.5843 - accuracy: 0.3837
Epoch 87/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.5861 - accuracy: 0.3692
Epoch 88/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.5660 - accuracy: 0.4197
Epoch 89/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.6126 - accuracy: 0.3594
Epoch 90/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.5933 - accuracy: 0.4009
Epoch 91/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.6207 - accuracy: 0.3558
Epoc

14/14 [==============================] - 0s 828us/step - loss: 1.4669 - accuracy: 0.4483
Epoch 163/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4680 - accuracy: 0.4112
Epoch 164/5000
14/14 [==============================] - 0s 813us/step - loss: 1.4910 - accuracy: 0.4321
Epoch 165/5000
14/14 [==============================] - 0s 890us/step - loss: 1.4747 - accuracy: 0.4327
Epoch 166/5000
14/14 [==============================] - 0s 969us/step - loss: 1.4925 - accuracy: 0.4142
Epoch 167/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.5086 - accuracy: 0.3935
Epoch 168/5000
14/14 [==============================] - 0s 937us/step - loss: 1.5153 - accuracy: 0.4101
Epoch 169/5000
14/14 [==============================] - 0s 940us/step - loss: 1.5029 - accuracy: 0.4061
Epoch 170/5000
14/14 [==============================] - 0s 956us/step - loss: 1.5500 - accuracy: 0.3997
Epoch 171/5000
14/14 [==============================] - 0s 981us/step - loss: 1.533

14/14 [==============================] - 0s 1ms/step - loss: 1.3802 - accuracy: 0.4459
Epoch 243/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4059 - accuracy: 0.4710
Epoch 244/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4407 - accuracy: 0.4247
Epoch 245/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4008 - accuracy: 0.4490
Epoch 246/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4492 - accuracy: 0.4295
Epoch 247/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4579 - accuracy: 0.4206
Epoch 248/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4817 - accuracy: 0.4072
Epoch 249/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4457 - accuracy: 0.4444
Epoch 250/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4206 - accuracy: 0.4371
Epoch 251/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4516 - accuracy: 0.

14/14 [==============================] - 0s 1ms/step - loss: 1.4272 - accuracy: 0.4096
Epoch 323/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3797 - accuracy: 0.4808
Epoch 324/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4159 - accuracy: 0.4563
Epoch 325/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.4224 - accuracy: 0.4641
Epoch 326/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3732 - accuracy: 0.4322
Epoch 327/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3430 - accuracy: 0.4737
Epoch 328/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3988 - accuracy: 0.4598
Epoch 329/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3942 - accuracy: 0.4547
Epoch 330/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3646 - accuracy: 0.5044
Epoch 331/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.3090 - accuracy: 0.

...

14/14 [==============================] - 0s 1ms/step - loss: 0.1506 - accuracy: 0.9569
Epoch 4672/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1714 - accuracy: 0.9434
Epoch 4673/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1457 - accuracy: 0.9493
Epoch 4674/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1524 - accuracy: 0.9543
Epoch 4675/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1383 - accuracy: 0.9561
Epoch 4676/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1514 - accuracy: 0.9579
Epoch 4677/5000
14/14 [==============================] - 0s 991us/step - loss: 0.1442 - accuracy: 0.9431
Epoch 4678/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1133 - accuracy: 0.9695
Epoch 4679/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1715 - accuracy: 0.9336
Epoch 4680/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1675 - a

14/14 [==============================] - 0s 1ms/step - loss: 0.1658 - accuracy: 0.9398
Epoch 4751/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1573 - accuracy: 0.9495
Epoch 4752/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.3188 - accuracy: 0.8581
Epoch 4753/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.4672 - accuracy: 0.8516
Epoch 4754/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.5116 - accuracy: 0.8384
Epoch 4755/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.4680 - accuracy: 0.8306
Epoch 4756/5000
14/14 [==============================] - 0s 1ms/step - loss: 2.4420 - accuracy: 0.5969
Epoch 4757/5000
14/14 [==============================] - 0s 1ms/step - loss: 1.1799 - accuracy: 0.6871
Epoch 4758/5000
14/14 [==============================] - 0s 2ms/step - loss: 1.1425 - accuracy: 0.7305
Epoch 4759/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.8108 - acc

14/14 [==============================] - 0s 1ms/step - loss: 0.2839 - accuracy: 0.8956
Epoch 4830/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1973 - accuracy: 0.9211
Epoch 4831/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.2219 - accuracy: 0.9197
Epoch 4832/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1773 - accuracy: 0.9553
Epoch 4833/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.2728 - accuracy: 0.9006
Epoch 4834/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.2074 - accuracy: 0.9292
Epoch 4835/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.2156 - accuracy: 0.9128
Epoch 4836/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1750 - accuracy: 0.9435
Epoch 4837/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1896 - accuracy: 0.9363
Epoch 4838/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1506 - acc

14/14 [==============================] - 0s 1ms/step - loss: 0.1635 - accuracy: 0.9422
Epoch 4909/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1873 - accuracy: 0.9234
Epoch 4910/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.2450 - accuracy: 0.9187
Epoch 4911/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.2335 - accuracy: 0.9173
Epoch 4912/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1346 - accuracy: 0.9592
Epoch 4913/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1221 - accuracy: 0.9599
Epoch 4914/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.2011 - accuracy: 0.9194
Epoch 4915/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1383 - accuracy: 0.9518
Epoch 4916/5000
14/14 [==============================] - 0s 1ms/step - loss: 0.1612 - accuracy: 0.9499
Epoch 4917/5000
14/14 [==============================] - 0s 955us/step - loss: 0.1459 - a

14/14 [==============================] - 0s 1ms/step - loss: 0.1219 - accuracy: 0.9606
Epoch 4988/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1041 - accuracy: 0.9754
Epoch 4989/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1360 - accuracy: 0.9388
Epoch 4990/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1468 - accuracy: 0.9615
Epoch 4991/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.2629 - accuracy: 0.8979
Epoch 4992/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1857 - accuracy: 0.9163
Epoch 4993/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1303 - accuracy: 0.9573
Epoch 4994/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1515 - accuracy: 0.9584
Epoch 4995/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1713 - accuracy: 0.9526
Epoch 4996/5000
14/14 [==============================] - 0s 2ms/step - loss: 0.1710 - acc